In [1]:
import os 
import sys
import numpy as np

sys.path.append("../submissions/keras_inceptionResNetV2_finetunning_imbalanced/")
sys.path.append("../")

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [3]:
from image_preprocessor import transform, transform_test
from batch_classifier import BatchClassifier, load_pretrained_model
from problem import get_cv, get_train_data, Predictions, score_types

Using TensorFlow backend.


In [4]:
from rampwf.workflows.image_classifier import _chunk_iterator

In [5]:
n_classes = 403

In [6]:
builder = BatchClassifier()
load_pretrained_model(builder.model, logs_path='/home/working_directory/ml/ramp-kits/pollenating_insects_3/logs_2017-10-03-11-17')

Load best loss weights:  /home/working_directory/ml/ramp-kits/pollenating_insects_3/logs_2017-10-03-11-17/weights/InceptionResNetV2_20_val_loss=1.7454_val_acc=0.5770_val_f170=0.0433.h5 1.7454


In [7]:
trained_model = transform, transform_test, builder

In [8]:
folder_X_array, y_train = get_train_data(path="..")

In [9]:
import cv2

In [99]:
test_batch_size = 1024

folder, X_array = folder_X_array
transform_img, transform_test_img, clf = trained_model
it = _chunk_iterator(X_array, folder=folder, chunk_size=test_batch_size)
y_proba = []
for X in it:
    for i in range(0, len(X), test_batch_size):
        # 1) Preprocessing
        X_batch = X[i: i + test_batch_size]
        # X_batch = Parallel(n_jobs=self.n_jobs, backend='threading')(
        #     delayed(transform_img)(x) for x in X_batch)
        X_batch = [transform_test_img(x) for x in X_batch]
        # X is a list of numpy arrrays at this point, convert it to a
        # single numpy array.
        try:
            X_batch = [x[np.newaxis, :, :, :] for x in X_batch]
        except IndexError:
            # single channel
            X_batch = [
                x[np.newaxis, np.newaxis, :, :] for x in X_batch]
        X_batch = np.concatenate(X_batch, axis=0)

        # 2) Prediction
        print("Start predictions...")
        X_batch_aug1 = np.zeros_like(X_batch)
        X_batch_aug2 = np.zeros_like(X_batch)
        for i, x in enumerate(X_batch):
            X_batch_aug1[i, ...] = cv2.flip(x, 0)
            X_batch_aug2[i, ...] = cv2.flip(x, 1)            
        y_proba_batch0 = clf.predict_proba(X_batch)        
        y_proba_batch1 = clf.predict_proba(X_batch_aug1)
        y_proba_batch2 = clf.predict_proba(X_batch_aug2)
        y_proba_batch = 0.33 * (y_proba_batch0 + y_proba_batch1 + y_proba_batch2)
        y_proba.append(y_proba_batch)
        break
    break
y_proba = np.concatenate(y_proba, axis=0)

Start predictions...


In [100]:
y_proba_batch.shape

(1024, 403)

In [80]:
for i in range(test_batch_size):
    print(np.max(y_proba_batch0[i, :]), np.argmax(y_proba_batch0[i, :]), y_proba_batch0[i, y_proba_batch0[i, :] > 0.5])

(0.13558371, 124, array([], dtype=float32))
(0.65769303, 159, array([ 0.65769303], dtype=float32))
(0.97289217, 117, array([ 0.97289217], dtype=float32))
(0.946141, 121, array([ 0.946141], dtype=float32))
(0.98147136, 108, array([ 0.98147136], dtype=float32))
(0.44682536, 27, array([], dtype=float32))
(0.80985063, 173, array([ 0.80985063], dtype=float32))
(0.63100082, 159, array([ 0.63100082], dtype=float32))
(0.26170903, 354, array([], dtype=float32))
(0.58912617, 180, array([ 0.58912617], dtype=float32))
(0.88288593, 57, array([ 0.88288593], dtype=float32))
(0.26484889, 180, array([], dtype=float32))
(0.25427717, 10, array([], dtype=float32))
(0.99758494, 39, array([ 0.99758494], dtype=float32))
(0.95805752, 159, array([ 0.95805752], dtype=float32))
(0.23291485, 3, array([], dtype=float32))
(0.99639744, 24, array([ 0.99639744], dtype=float32))
(0.69192034, 53, array([ 0.69192034], dtype=float32))
(0.46601364, 131, array([], dtype=float32))
(0.11483695, 89, array([], dtype=float32))
(

In [81]:
for i in range(test_batch_size):
    print(np.max(y_proba_batch1[i, :]), np.argmax(y_proba_batch1[i, :]), y_proba_batch1[i, y_proba_batch1[i, :] > 0.5])

(0.25138244, 124, array([], dtype=float32))
(0.58950537, 159, array([ 0.58950537], dtype=float32))
(0.9938215, 117, array([ 0.9938215], dtype=float32))
(0.96411043, 121, array([ 0.96411043], dtype=float32))
(0.97411668, 108, array([ 0.97411668], dtype=float32))
(0.32700714, 27, array([], dtype=float32))
(0.75069684, 173, array([ 0.75069684], dtype=float32))
(0.61738271, 202, array([ 0.61738271], dtype=float32))
(0.39136165, 72, array([], dtype=float32))
(0.39602894, 75, array([], dtype=float32))
(0.72648162, 180, array([ 0.72648162], dtype=float32))
(0.70685571, 75, array([ 0.70685571], dtype=float32))
(0.17414325, 66, array([], dtype=float32))
(0.99930155, 39, array([ 0.99930155], dtype=float32))
(0.99939024, 159, array([ 0.99939024], dtype=float32))
(0.60240227, 14, array([ 0.60240227], dtype=float32))
(0.99935836, 24, array([ 0.99935836], dtype=float32))
(0.92423928, 179, array([ 0.92423928], dtype=float32))
(0.34913734, 131, array([], dtype=float32))
(0.1773655, 89, array([], dtype

In [83]:
for i in range(test_batch_size):
    print(np.max(y_proba_batch2[i, :]), np.argmax(y_proba_batch2[i, :]), y_proba_batch2[i, y_proba_batch2[i, :] > 0.5])

(0.26075676, 170, array([], dtype=float32))
(0.55470043, 159, array([ 0.55470043], dtype=float32))
(0.99570566, 117, array([ 0.99570566], dtype=float32))
(0.92435586, 121, array([ 0.92435586], dtype=float32))
(0.99238634, 108, array([ 0.99238634], dtype=float32))
(0.45392922, 27, array([], dtype=float32))
(0.4533065, 173, array([], dtype=float32))
(0.86850792, 159, array([ 0.86850792], dtype=float32))
(0.55840087, 354, array([ 0.55840087], dtype=float32))
(0.54174972, 57, array([ 0.54174972], dtype=float32))
(0.94367784, 57, array([ 0.94367784], dtype=float32))
(0.67135626, 134, array([ 0.67135626], dtype=float32))
(0.2004077, 10, array([], dtype=float32))
(0.99574465, 39, array([ 0.99574465], dtype=float32))
(0.95945847, 159, array([ 0.95945847], dtype=float32))
(0.62639934, 3, array([ 0.62639934], dtype=float32))
(0.95680743, 24, array([ 0.95680743], dtype=float32))
(0.72008008, 53, array([ 0.72008008], dtype=float32))
(0.63978237, 131, array([ 0.63978237], dtype=float32))
(0.4636986

In [84]:
for i in range(batch_size):
    print(np.max(y_proba_batch[i, :]), np.argmax(y_proba_batch[i, :]), y_proba_batch[i, y_proba_batch[i, :] > 0.5])

(0.15893376, 124, array([], dtype=float32))
(0.59462661, 159, array([ 0.59462661], dtype=float32))
(0.97759837, 117, array([ 0.97759837], dtype=float32))
(0.93542039, 121, array([ 0.93542039], dtype=float32))
(0.97283161, 108, array([ 0.97283161], dtype=float32))
(0.40516138, 27, array([], dtype=float32))
(0.66457188, 173, array([ 0.66457188], dtype=float32))
(0.6176796, 159, array([ 0.6176796], dtype=float32))
(0.27416077, 354, array([], dtype=float32))
(0.38665015, 57, array([], dtype=float32))
(0.68579966, 57, array([ 0.68579966], dtype=float32))
(0.29989338, 134, array([], dtype=float32))
(0.17916735, 10, array([], dtype=float32))
(0.98756832, 39, array([ 0.98756832], dtype=float32))
(0.96257913, 159, array([ 0.96257913], dtype=float32))
(0.28396818, 3, array([], dtype=float32))


In [87]:
y_train[:test_batch_size]

array([124, 159, 117, 121, 108,  27, 173, 159, 153,  57,  57, 149,  72,
        39, 159, 102,  24, 179,  78, 134, 119, 122, 159, 151, 171, 202,
       159, 134,  77,  57,  72, 202,  61,  22, 149, 172, 155,  50, 172,
        76,  66,   8,   5, 159,   0,   0,  75, 165, 202,  84, 195, 175,
       202, 202, 215, 244, 110, 188, 179, 202, 202, 169, 150, 388, 202,
       159, 128, 131,  94, 202,  94, 179, 180, 159, 131, 159, 202,  57,
       332,  26, 202, 166, 136, 125, 134, 131,  20,  77, 159,  19, 173,
       381,  10,  68, 108,  97, 121, 313,  55,  14,  61,  57,  27, 202,
        71, 167, 202, 159, 205,  20, 131, 186,  93, 202,  54, 191,  78,
       190, 124, 163,  94,  57, 202,   5,  76, 200,  78, 159])

In [88]:
np.argmax(y_proba_batch, axis=1)

array([124, 159, 117, 121, 108,  27, 173, 159, 354,  57,  57, 134,  10,
        39, 159,   3,  24,  53, 131, 150, 119, 122, 159, 151, 171, 202,
       159, 134,  77,  57,  72, 202,  61,  22,  27, 172, 155,  15, 172,
        18, 112,   8,   5, 159,   0,   0,  75, 165, 202,  84, 195, 175,
       202, 202, 215, 244, 110, 188, 179, 202, 202, 169, 150,  25, 202,
         7, 128,  18,  94, 202, 100, 179, 180,  51, 131, 159, 202,  57,
        91,  26, 202,   3, 136, 125,  77, 131,  20,  77, 159, 152, 173,
        11,  10,  68, 108,  97, 189, 313, 148,  14,  61,  57,  27, 202,
        71, 202, 202,   1,  92,  20, 188, 186,  93, 202, 203, 191,  78,
       190, 124, 163,  94,  57, 202,   5, 111,   9,  39, 159])

In [89]:
from sklearn.metrics import fbeta_score

In [101]:
y_true_labels = y_train[:test_batch_size]

y_pred_labels0 = np.argmax(y_proba_batch0, axis=1)
y_pred_labels1 = np.argmax(y_proba_batch, axis=1)

In [102]:
res1 = fbeta_score(y_true_labels, y_pred_labels0, beta=1, average=None)
res2 = fbeta_score(y_true_labels, y_pred_labels1, beta=1, average=None)
res1, res2

(array([ 0.8       ,  0.66666667,  0.        ,  0.76190476,  0.        ,
         0.92307692,  0.66666667,  0.57142857,  0.        ,  0.4       ,
         0.        ,  0.28571429,  0.92682927,  1.        ,  0.        ,
         0.54545455,  0.66666667,  0.91666667,  0.85714286,  1.        ,
         1.        ,  0.85714286,  0.5       ,  0.51612903,  0.        ,
         1.        ,  0.        ,  0.66666667,  0.33333333,  0.83333333,
         1.        ,  0.5       ,  1.        ,  0.69565217,  1.        ,
         0.6       ,  0.88888889,  0.        ,  1.        ,  1.        ,
         0.66666667,  1.        ,  0.        ,  0.4       ,  0.5       ,
         0.57142857,  0.        ,  0.42857143,  0.5       ,  0.5       ,
         0.75609756,  0.        ,  0.8       ,  0.        ,  1.        ,
         0.85714286,  1.        ,  0.66666667,  0.        ,  0.25      ,
         0.85714286,  1.        ,  0.8       ,  0.75      ,  1.        ,
         0.5       ,  0.57142857,  0.85714286,  0.5

In [103]:
len(res1[res1>0.7]) * 1.0 / len(res1) 

0.4048582995951417

In [104]:
len(res2[res2>0.7]) * 1.0 / len(res2) 

0.47540983606557374

In [ ]:
!

In [12]:
from problem import workflow
workflow.n_jobs = 10
workflow.chunk_size = 256
workflow.test_batch_size = 128

In [13]:
workflow.test_submission(trained_model, folder_X_array)

KeyboardInterrupt: 